<a href="https://colab.research.google.com/github/wiktormorawski/Machine-Learning-Projects/blob/master/Kopia_notatnika_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import PIL

In [ ]:
print('Pillow Version:', PIL.__version__)
print('TF Version:', tf.__version__)

Pillow Version: 7.1.2
TF Version: 2.5.0


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"wiktormorawski","key":"acaff2ace22eaa8e808e3c119f35611d"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d puneet6060/intel-image-classification -p /content/intel/ --unzip

 95% 330M/346M [00:03<00:00, 102MB/s]
100% 346M/346M [00:03<00:00, 100MB/s]


In [ ]:
from skimage.io import imread
from skimage.transform import resize
import os
## są posortowane wedle building,forest ...
def check_size(images):
  l = []
  for i in images:
    size = i.shape
    if size not in l:
      l.append(size)
  return l
## Podział na testowy i treningowy oraz resize i zmiana na szarość
def load_images_from_folder(type):
    images = []
    labels = []
    if type == 'train':
      labels = ['/content/intel/seg_train/seg_train/buildings', '/content/intel/seg_train/seg_train/forest', '/content/intel/seg_train/seg_train/glacier', '/content/intel/seg_train/seg_train/mountain', '/content/intel/seg_train/seg_train/sea', '/content/intel/seg_train/seg_train/street']
    if type == 'test':
      labels = ['/content/intel/seg_test/seg_test/buildings', '/content/intel/seg_test/seg_test/forest', '/content/intel/seg_test/seg_test/glacier', '/content/intel/seg_test/seg_test/mountain', '/content/intel/seg_test/seg_test/sea', '/content/intel/seg_test/seg_test/street']

    lab = []
    count = 0
    for i in labels:
      for filename in os.listdir(i):
          img = imread(os.path.join(i,filename),as_gray=True)
          img = np.resize(img, (64,64))
          if img is not None:
            lab.append(count)
            images.append(np.array(img))
      count += 1
    return np.array(images, dtype=object), lab
  

images_train, labels_train = load_images_from_folder('train')
images_test, labels_test = load_images_from_folder('test')
print("Wielkosc Obrazów ogółem TRAIN: ", images_train.shape[0])
print("labels TRAIN: ", labels_train)
print("Wielkosc Obrazów ogółem TEST: ", images_test.shape[0])
print("labels TEST: ", labels_test)

Wielkosc Obrazów ogółem TRAIN:  14034
labels TRAIN:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print("Liczba klas: ", len(np.unique(labels_train)))

Liczba klas:  6


In [ ]:
## Przetasowanie
def shuffle_images_and_labels(a_images, a_labels):
    shuffler = np.random.permutation(len(a_images))
    array1_shuffled = a_images[shuffler]
    array2_shuffled = np.array(a_labels)[shuffler]
    return array1_shuffled,array2_shuffled
images_train, labels_train = shuffle_images_and_labels(images_train, labels_train)
print(images_train[0], labels_train[0])

print('Size of images: ', check_size(images_train))

[[0.7593874509803922 0.7450999999999999 0.7501525490196078 ...
  0.7789952941176471 0.7829168627450981 0.7868384313725492]
 [0.7868384313725492 0.7868384313725492 0.7868384313725492 ...
  0.8714898039215687 0.9454419607843138 0.9678403921568627]
 [0.952881568627451 0.9881905882352943 0.9733752941176471 ...
  0.7742403921568627 0.7624756862745099 0.7624756862745099]
 ...
 [0.71604 0.669569411764706 0.29957294117647065 ... 0.7053109803921569
  0.7014192156862745 0.6874074509803921]
 [0.7624380392156863 0.6649117647058824 0.5965705882352942 ...
  0.6645564705882353 0.609356862745098 0.7045380392156863]
 [0.7072682352941178 0.761001568627451 0.7147988235294117 ...
  0.7193007843137255 0.7938105882352943 0.7265784313725491]] 2
Size of images:  [(64, 64)]


In [ ]:
## Procentowy udział klas w zbiorach

def appear_in_class(labels_train, labels_test):
  result1 = []
  result2 = []
  for i in range(6):
    count = 0
    for j in labels_train:
      if j == i:
        count += 1
    result1.append(count)
  for i in range(6):
    for j in labels_test:
      if j == i:
        count += 1
    result2.append(count)
  return result1, result2
labels = ["Buildings: ", "Forest: ", "Glacier: ", "Mountain: ", "Sea: ", "Street: "]
results_appear_train, results_appear_test = appear_in_class(labels_train,labels_test)
